In [ ]:
import os, random, numpy as np
import tensorflow as tf
import keras_tuner as kt
import matplotlib
import matplotlib.pyplot as plt

import utils  # your local utils.py

from sklearn.utils import class_weight
from tensorflow.keras import mixed_precision

# Set up reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

# Use mixed precision and GPU memory management like tiny.py
mixed_precision.set_global_policy('mixed_float16')
matplotlib.use("Agg")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_memory_growth(gpus[0], True)


In [ ]:
IMG_SIZE = 256
BATCH = 2

TRAIN_CSV = "./split_data/train.csv"
VAL_CSV = "./split_data/val.csv"
TEST_CSV = "./split_data/test.csv"

train_paths, train_labels, train_label_encoder = utils.load_paths_and_labels(TRAIN_CSV)
val_paths, val_labels, val_label_encoder = utils.load_paths_and_labels(VAL_CSV)
test_paths, test_labels, test_label_encoder = utils.load_paths_and_labels(TEST_CSV)

# Optional: SAMPLE tiny dataset for FAST testing
# train_paths, train_labels = train_paths[:20], train_labels[:20]
# val_paths, val_labels = val_paths[:10], val_labels[:10]

# Show class distributions
utils.show_dataset_class_distribution("TRAIN", train_labels)
utils.show_dataset_class_distribution("VAL  ", val_labels)
utils.show_dataset_class_distribution("TEST ", test_labels)

# Build datasets
augment_layer = utils.make_augment()
train_ds = utils.make_dataset(train_paths, train_labels, BATCH, shuffle=True, autotune=True)
train_ds = train_ds.map(lambda imgs, labs: (augment_layer(imgs, training=True), labs), num_parallel_calls=tf.data.AUTOTUNE)
val_ds = utils.make_dataset(val_paths, val_labels, BATCH, shuffle=False, autotune=True)


In [ ]:
cw_vals = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_w = dict(enumerate(cw_vals))


In [ ]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3)))
    
    for i in range(hp.Int('conv_blocks', 2, 3, default=3)):
        filters = hp.Choice(f'filters_{i}', values=[32, 64, 128], default=64)
        model.add(tf.keras.layers.Conv2D(filters, (3,3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(2,2))
    
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(hp.Float('dropout_flat', 0.4, 0.6, step=0.1, default=0.5)))
    model.add(tf.keras.layers.Dense(hp.Int('dense_units', 64, 192, step=32, default=128), activation='relu'))
    model.add(tf.keras.layers.Dropout(hp.Float('dropout_dense', 0.4, 0.6, step=0.1, default=0.5)))
    model.add(tf.keras.layers.Dense(len(train_label_encoder.classes_), activation='softmax'))
    
    lr = hp.Float('learning_rate', 1e-4, 5e-4, sampling='log', default=1e-4)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=['accuracy'])
    return model


In [ ]:
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=5,   # FAST: reduce for tests
    factor=3,
    directory='logs',
    project_name='RealWaste_quick_test'
)


In [ ]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir='logs/tensorboard', histogram_freq=1)
tuner.search(
    train_ds,
    validation_data=val_ds,
    class_weight=class_w,
    callbacks=[
        tensorboard_cb,
        utils.checkpoint_cb(),
        utils.csv_logger_cb()
    ]
)


In [ ]:
best_hp = tuner.get_best_hyperparameters(1)[0]
print("Best hyperparameters found:")
for k, v in best_hp.values.items():
    print(f"{k}: {v}")

best_model = tuner.get_best_models(1)[0]
best_model.save("best_tuned_trashnet_model.keras")
